In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 849kB 5.1MB/s 
     |████████████████████████████████| 184kB 10.5MB/s 
     |████████████████████████████████| 276kB 12.4MB/s 
     |████████████████████████████████| 112kB 16.1MB/s 
     |████████████████████████████████| 829kB 14.9MB/s 
     |████████████████████████████████| 1.3MB 21.3MB/s 
     |████████████████████████████████| 143kB 36.4MB/s 
     |████████████████████████████████| 296kB 17.7MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=0fd78465a8969fe672f5f5aae3e23d4c1566a0c5e4067798c5cc6bd69d7bf4c2
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=03be69ad853363e62204542de4769bf3530c7d59f66c1f2ce9a1d84962b1420f
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAML

In [ ]:
import json
import gzip
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from tqdm import tqdm


### Reading from a zip file

In [ ]:

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)
a=parse('All_Beauty_5.json.gz')

### Import the dataset

In [ ]:
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('All_Beauty_5.json.gz')

### Using the meta data to get the product details 

In [ ]:
# Load all data
meta = []
with open('meta_All_Beauty.json', 'r') as f:
    for l in tqdm(f):
        meta.append(json.loads(l)) 


meta_hash = {}
for m in meta:
    keys = list(m['details'].keys())
    for key in keys:
        if key.startswith('ASIN: '):
            if m['details'][key] not in meta_hash:
                meta_hash[m['details'][key]] = m



# get product from product id
def get_product(id: str):
    item = meta_hash[id]
    new_item  = to_product(item)
    return new_item


# product formatter
def to_product(item):
    new_item = {}
    fields = ['title', 'brand', 'description', 'main_cat']
    for f in fields:
        new_item[f] = item[f]
    return new_item


get_product('B01HJEGTYK')

32892it [00:00, 37453.52it/s]


{'brand': '',
 'description': [],
 'main_cat': 'All Beauty',
 'title': 'FESHFEN Scrunchy Scrunchies Synthetic Hair Bun Updo Hair Pieces Ponytail Hair Extensions Wavy Curly Messy Hair Bun Extensions Donut Hair Chignons Piece Wig'}

In [ ]:
df.tail(20)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
5249,5.0,True,"04 19, 2016",AGXW8BFME8048,B00VG1AV5Q,NaN,Gretchen G,This is a great product. It is alcohol free bu...,This is a great product. It is alcohol free bu...,1461024000,NaN,NaN
5250,5.0,True,"03 13, 2016",A29WQLB8ACNZ5T,B00VG1AV5Q,NaN,Marie Buckley,I use this product daily I love it.,Five Stars,1457827200,NaN,NaN
5251,5.0,True,"03 2, 2016",A3980XKDL0SZEX,B00VG1AV5Q,NaN,Amazon Customer,Very satisfied.,Five Stars,1456876800,NaN,NaN
5252,5.0,True,"02 25, 2016",A34OON6ZYATX36,B00VG1AV5Q,NaN,faye,I have very curly hair and always used Mousse ...,Works as advertised.,1456358400,NaN,NaN
5253,5.0,True,"01 5, 2017",A2T6UI6V98KT94,B019LAI4HU,NaN,Joi Sanders,This is one of the most beautiful lotions I ha...,Ultimate Kors,1483574400,NaN,NaN
5254,5.0,True,"05 6, 2016",A2KV46HMWY1YWB,B019LAI4HU,NaN,Kindle Customer,Smells wonderful. Feels great on hands and arm...,Must have product,1462492800,NaN,NaN
5255,5.0,False,"05 5, 2016",A3ETTJOVI6C9V5,B019LAI4HU,NaN,W. Keane,"I LOVE the smell, the texture, everything abou...",I LOVE the smell,1462406400,NaN,NaN
5256,5.0,False,"03 14, 2016",A1NP1AD8FGSA71,B019LAI4HU,NaN,mercedes,I get more compliments when I wear this lotion!!!,Very Clean Scent,1457913600,NaN,NaN
5257,5.0,True,"01 24, 2016",A1MDYWL5DLS9SS,B019LAI4HU,NaN,Deb Houghtaling,I love this lotion. It has a light clean smel...,Best Ever,1453593600,NaN,NaN
5258,2.0,True,"03 27, 2018",A2TU781PWGS09X,B019V2KYZS,NaN,Amazon Customer,Doesnt smell,Two Stars,1522108800,NaN,NaN


In [ ]:
df.head(10)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"09 1, 2016",A3CIUOJXQ5VDQ2,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Shelly F,As advertised. Reasonably priced,Five Stars,1472688000,NaN,NaN
1,5.0,True,"11 14, 2013",A3H7T87S984REU,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",houserules18,Like the oder and the feel when I put it on my...,Good for the face,1384387200,NaN,NaN
2,1.0,True,"08 18, 2013",A3J034YH7UG4KT,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Adam,I bought this to smell nice after I shave. Wh...,Smells awful,1376784000,NaN,NaN
3,5.0,False,"05 3, 2011",A2UEO5XR3598GI,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,Truth is There IS Nothing Like an AQUA VELVA MAN.,1304380800,25,NaN
4,5.0,True,"05 6, 2011",A3SFRT223XXWF7,B00006L9LC,{'Size:': ' 200ml/6.7oz'},C. C. Christian,If you ever want to feel pampered by a shampoo...,Bvlgari Shampoo,1304640000,3,NaN
5,5.0,False,"05 16, 2010",A24HQ2N7332W7W,B00006L9LC,{'Size:': ' 366'},Kindle Customer Joyce Wilson,"If you know the scent of Diva, you'll LOVE thi...",Diva is Heavenly,1273968000,NaN,NaN
6,5.0,False,"05 7, 2018",A2G90R2ZU6KU5D,B00006L9LC,{'Size:': ' Small'},Mike,Got this shampoo as a solution for my wife's d...,"Outstanding, no complains",1525651200,NaN,NaN
7,2.0,True,"05 7, 2018",A24W4W9E62FZP2,B00006L9LC,{'Size:': ' Small'},Reb,No change my scalp still itches like crazy. It...,No change my scalp still itches like crazy. It...,1525651200,NaN,NaN
8,1.0,True,"05 6, 2018",A7ID5H7FWLJHC,B00006L9LC,{'Size:': ' Small'},U. V.,Too expensive for such poor quality. There was...,Too expensive for such poor quality. There was...,1525564800,NaN,NaN
9,1.0,True,"05 6, 2018",AYKOSAJTP5AVS,B00006L9LC,{'Size:': ' Small'},Senthil Kumar M,"It dries my hair, doesnt help to reduce dandru...","Dries my hair, doesnt help to reduce dandruff....",1525564800,NaN,NaN


In [ ]:
# adding time value to the reviewTime to make the data unique as the the original reviewTime has the same date value for all the products of a particular customer
df['reviewTime'] = pd.to_datetime(df['reviewTime'])
hrs = pd.Timedelta(hours=9)
min = pd.to_timedelta(df.groupby('reviewTime').cumcount().add(1).mul(7), unit='s')
df['reviewTime']=df['reviewTime'] + hrs + min 

In [ ]:
df['overall'] = torch.tensor([int(f) for f in df['overall']], dtype=torch.long)

In [ ]:
df.head(20)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,True,2016-09-01 09:00:07,A3CIUOJXQ5VDQ2,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Shelly F,As advertised. Reasonably priced,Five Stars,1472688000,NaN,NaN
1,5,True,2013-11-14 09:00:07,A3H7T87S984REU,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",houserules18,Like the oder and the feel when I put it on my...,Good for the face,1384387200,NaN,NaN
2,1,True,2013-08-18 09:00:07,A3J034YH7UG4KT,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Adam,I bought this to smell nice after I shave. Wh...,Smells awful,1376784000,NaN,NaN
3,5,False,2011-05-03 09:00:07,A2UEO5XR3598GI,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,Truth is There IS Nothing Like an AQUA VELVA MAN.,1304380800,25,NaN
4,5,True,2011-05-06 09:00:07,A3SFRT223XXWF7,B00006L9LC,{'Size:': ' 200ml/6.7oz'},C. C. Christian,If you ever want to feel pampered by a shampoo...,Bvlgari Shampoo,1304640000,3,NaN
5,5,False,2010-05-16 09:00:07,A24HQ2N7332W7W,B00006L9LC,{'Size:': ' 366'},Kindle Customer Joyce Wilson,"If you know the scent of Diva, you'll LOVE thi...",Diva is Heavenly,1273968000,NaN,NaN
6,5,False,2018-05-07 09:00:07,A2G90R2ZU6KU5D,B00006L9LC,{'Size:': ' Small'},Mike,Got this shampoo as a solution for my wife's d...,"Outstanding, no complains",1525651200,NaN,NaN
7,2,True,2018-05-07 09:00:14,A24W4W9E62FZP2,B00006L9LC,{'Size:': ' Small'},Reb,No change my scalp still itches like crazy. It...,No change my scalp still itches like crazy. It...,1525651200,NaN,NaN
8,1,True,2018-05-06 09:00:07,A7ID5H7FWLJHC,B00006L9LC,{'Size:': ' Small'},U. V.,Too expensive for such poor quality. There was...,Too expensive for such poor quality. There was...,1525564800,NaN,NaN
9,1,True,2018-05-06 09:00:14,AYKOSAJTP5AVS,B00006L9LC,{'Size:': ' Small'},Senthil Kumar M,"It dries my hair, doesnt help to reduce dandru...","Dries my hair, doesnt help to reduce dandruff....",1525564800,NaN,NaN


In [ ]:
# Here the reviewerID is filtered according to reviewTime, we are considering this to split our data into train and test. 
#Test will have the recent timestamp value data and the train will have everything other than the recent timestamp.
df['rank_latest']= df.groupby(['reviewerID'])['reviewTime'].rank(method='first', ascending=False)

In [ ]:
df.head(20)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,rank_latest
0,5,True,2016-09-01 09:00:07,A3CIUOJXQ5VDQ2,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Shelly F,As advertised. Reasonably priced,Five Stars,1472688000,NaN,NaN,2.0
1,5,True,2013-11-14 09:00:07,A3H7T87S984REU,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",houserules18,Like the oder and the feel when I put it on my...,Good for the face,1384387200,NaN,NaN,5.0
2,1,True,2013-08-18 09:00:07,A3J034YH7UG4KT,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Adam,I bought this to smell nice after I shave. Wh...,Smells awful,1376784000,NaN,NaN,5.0
3,5,False,2011-05-03 09:00:07,A2UEO5XR3598GI,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,Truth is There IS Nothing Like an AQUA VELVA MAN.,1304380800,25,NaN,2.0
4,5,True,2011-05-06 09:00:07,A3SFRT223XXWF7,B00006L9LC,{'Size:': ' 200ml/6.7oz'},C. C. Christian,If you ever want to feel pampered by a shampoo...,Bvlgari Shampoo,1304640000,3,NaN,5.0
5,5,False,2010-05-16 09:00:07,A24HQ2N7332W7W,B00006L9LC,{'Size:': ' 366'},Kindle Customer Joyce Wilson,"If you know the scent of Diva, you'll LOVE thi...",Diva is Heavenly,1273968000,NaN,NaN,4.0
6,5,False,2018-05-07 09:00:07,A2G90R2ZU6KU5D,B00006L9LC,{'Size:': ' Small'},Mike,Got this shampoo as a solution for my wife's d...,"Outstanding, no complains",1525651200,NaN,NaN,5.0
7,2,True,2018-05-07 09:00:14,A24W4W9E62FZP2,B00006L9LC,{'Size:': ' Small'},Reb,No change my scalp still itches like crazy. It...,No change my scalp still itches like crazy. It...,1525651200,NaN,NaN,5.0
8,1,True,2018-05-06 09:00:07,A7ID5H7FWLJHC,B00006L9LC,{'Size:': ' Small'},U. V.,Too expensive for such poor quality. There was...,Too expensive for such poor quality. There was...,1525564800,NaN,NaN,5.0
9,1,True,2018-05-06 09:00:14,AYKOSAJTP5AVS,B00006L9LC,{'Size:': ' Small'},Senthil Kumar M,"It dries my hair, doesnt help to reduce dandru...","Dries my hair, doesnt help to reduce dandruff....",1525564800,NaN,NaN,5.0


**Converting the user and item data into int values as the string ids will not be accepted by the model**

In [ ]:
df['reviewerID'] = df['reviewerID'].astype('category')
df['asin'] = df['asin'].astype('category')
d = dict(enumerate(df['reviewerID'].cat.categories))
e = dict(enumerate(df['asin'].cat.categories))
df['reviewerID'] = df['reviewerID'].cat.codes
df['asin'] = df['asin'].cat.codes

In [ ]:
df.head(10)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,rank_latest
0,5,True,2016-09-01 09:00:07,610,0,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Shelly F,As advertised. Reasonably priced,Five Stars,1472688000,NaN,NaN,2.0
1,5,True,2013-11-14 09:00:07,644,0,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",houserules18,Like the oder and the feel when I put it on my...,Good for the face,1384387200,NaN,NaN,5.0
2,1,True,2013-08-18 09:00:07,663,0,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Adam,I bought this to smell nice after I shave. Wh...,Smells awful,1376784000,NaN,NaN,5.0
3,5,False,2011-05-03 09:00:07,489,0,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,Truth is There IS Nothing Like an AQUA VELVA MAN.,1304380800,25,NaN,2.0
4,5,True,2011-05-06 09:00:07,731,1,{'Size:': ' 200ml/6.7oz'},C. C. Christian,If you ever want to feel pampered by a shampoo...,Bvlgari Shampoo,1304640000,3,NaN,5.0
5,5,False,2010-05-16 09:00:07,280,1,{'Size:': ' 366'},Kindle Customer Joyce Wilson,"If you know the scent of Diva, you'll LOVE thi...",Diva is Heavenly,1273968000,NaN,NaN,4.0
6,5,False,2018-05-07 09:00:07,374,1,{'Size:': ' Small'},Mike,Got this shampoo as a solution for my wife's d...,"Outstanding, no complains",1525651200,NaN,NaN,5.0
7,2,True,2018-05-07 09:00:14,283,1,{'Size:': ' Small'},Reb,No change my scalp still itches like crazy. It...,No change my scalp still itches like crazy. It...,1525651200,NaN,NaN,5.0
8,1,True,2018-05-06 09:00:07,784,1,{'Size:': ' Small'},U. V.,Too expensive for such poor quality. There was...,Too expensive for such poor quality. There was...,1525564800,NaN,NaN,5.0
9,1,True,2018-05-06 09:00:14,980,1,{'Size:': ' Small'},Senthil Kumar M,"It dries my hair, doesnt help to reduce dandru...","Dries my hair, doesnt help to reduce dandruff....",1525564800,NaN,NaN,5.0


In [ ]:
# Here we are splitting the data according to the above timestamp condition 
train_data = df[df['rank_latest'] != 1]
test_data = df[df['rank_latest'] == 1]

# drop columns that we no longer need
train_data = train_data[['reviewerID', 'asin', 'overall']]
test_data = test_data[['reviewerID', 'asin', 'overall']]

In [ ]:
len(train_data)

4278

In [ ]:
len(test_data)

991

In [ ]:
train_data.tail(20)

,reviewerID,asin,overall
5111,806,5,2
5112,378,5,5
5113,968,5,5
5114,107,5,5
5116,932,6,3
5125,400,11,5
5126,932,13,3
5127,932,13,5
5132,932,29,3
5177,418,57,4


In [ ]:
# We will only consider implicit ratings here from the above explicit ones. Thus, converting all the ratings of the user with that item as 1. 
# Where 1 represents that the user has interacted with that item.
train_data.loc[:, 'overall'] = 1

In [ ]:
train_data.head(20)

,reviewerID,asin,overall
0,610,0,1
1,644,0,1
2,663,0,1
3,489,0,1
4,731,1,1
5,280,1,1
6,374,1,1
7,283,1,1
8,784,1,1
9,980,1,1


**We also require negative samples to train our models, to indicate items that the user has not interacted with. We assume that such items are those that the user are not interested in. For now we randomly assign 5 such negative samples for each user-item pair.**

In [ ]:
# Get a list of all product IDs
all_pids = df['asin'].unique()

# Pointers that will hold the training data
users, items, labels = [], [], []

# This is the set of items that each user has interaction with
user_item_set = set(zip(train_data['reviewerID'], train_data['asin']))

# 5:1 ratio of negative to positive samples
num_negatives = 5

for (u, i) in user_item_set:
    users.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_pids) 
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_pids)
        users.append(u)
        items.append(negative_item)
        labels.append(0) # items not interacted with are negative

In [ ]:
print(users[:9])
print(items[:9])
print(labels[:9])

[285, 285, 285, 285, 285, 285, 31, 31, 31]
[81, 71, 64, 66, 32, 53, 6, 81, 48]
[1, 0, 0, 0, 0, 0, 1, 0, 0]


**The class below simply encapsulates the code we have written above into a PyTorch Dataset class.**

In [ ]:
class AmazonTrainDataset(Dataset):

    def __init__(self, df, all_pids):
        self.users, self.items, self.labels = self.get_dataset(df, all_pids)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, df, all_pids):
        users, items, labels = [], [], []
        user_item_set = set(zip(df['reviewerID'], df['asin']))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_pids)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_pids)
                users.append(u)
                items.append(negative_item)
                labels.append(0)

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

**NCF model using PyTorch Lightning by taking the user and item vectors, which are concatenated and passed to a fully connected layer. Here the optimizer is adam**

In [ ]:
class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the product ratings for training
            all_pids (list): List containing all item ids (train + test)
    """
    
    def __init__(self, num_users, num_items, df, all_pids):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.df = df
        self.all_pids = all_pids
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(AmazonTrainDataset(self.df, self.all_pids),
                          batch_size=512, num_workers=4)

**Train the model for 10 epochs**

In [ ]:
num_users = len(df['reviewerID'])
num_items = len(df['asin'])
all_pids = df['asin'].unique()

model = NCF(num_users, num_items, train_data, all_pids)

trainer = pl.Trainer(max_epochs=200, gpus=1, reload_dataloaders_every_epoch=True,
                     progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 42.2 K
1 | item_embedding | Embedding | 42.2 K
2 | fc1            | Linear    | 1.1 K 
3 | fc2            | Linear    | 2.1 K 
4 | output         | Linear    | 33    
---------------------------------------------
87.5 K    Trainable params
0         Non-trainable params
87.5 K    Total params
0.350     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze

1

In [ ]:
# User-item pairs for testing
test_user_item_set = set(zip(test_data['reviewerID'], test_data['asin']))

# Dict of all items that are interacted with by each user
user_interacted_items = df.groupby('reviewerID')['asin'].apply(list).to_dict()

hits = []
for (u,i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_pids) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
  
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    #print("Top 10 recommended items for user: ",u,"is :",top10_items)
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))

The Hit Ratio @ 10 is 0.82


In [ ]:
test_user_item_set

{(0, 29),
 (1, 29),
 (2, 78),
 (3, 29),
 (4, 40),
 (5, 74),
 (6, 29),
 (7, 40),
 (8, 40),
 (9, 83),
 (10, 75),
 (11, 56),
 (12, 40),
 (13, 40),
 (14, 40),
 (15, 40),
 (16, 40),
 (17, 29),
 (18, 40),
 (19, 62),
 (20, 29),
 (21, 40),
 (22, 29),
 (23, 40),
 (24, 29),
 (25, 29),
 (26, 40),
 (27, 40),
 (28, 29),
 (29, 29),
 (30, 29),
 (31, 29),
 (32, 40),
 (33, 40),
 (34, 56),
 (35, 29),
 (36, 35),
 (37, 29),
 (38, 40),
 (39, 40),
 (40, 75),
 (41, 29),
 (42, 40),
 (43, 64),
 (44, 75),
 (45, 29),
 (46, 40),
 (47, 29),
 (48, 40),
 (49, 40),
 (50, 40),
 (51, 40),
 (52, 29),
 (53, 29),
 (54, 6),
 (55, 40),
 (56, 70),
 (57, 40),
 (58, 40),
 (59, 40),
 (60, 40),
 (61, 42),
 (62, 29),
 (63, 75),
 (64, 29),
 (65, 29),
 (66, 75),
 (67, 29),
 (68, 29),
 (69, 40),
 (70, 29),
 (71, 52),
 (72, 1),
 (73, 40),
 (74, 56),
 (75, 29),
 (76, 29),
 (77, 40),
 (78, 23),
 (79, 40),
 (80, 40),
 (81, 71),
 (82, 40),
 (83, 40),
 (84, 70),
 (85, 40),
 (86, 40),
 (87, 29),
 (88, 29),
 (89, 40),
 (90, 29),
 (91, 40),


In [ ]:
top10_item_title = []
for (u,i) in [(974, 40)]:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_pids) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
  
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    for j in top10_items:
      try:
        top10_item_title.append(get_product(test_data[test_data['asin']==j]['asin'].map(e).iloc[0])['title'])
      except:
        top10_item_title.append(j)
    print("Actual item bought by user: ",get_product(test_data[test_data['asin']==i]['asin'].map(e).iloc[0])['title'])
    print("Top 10 recommended items for user: ",u)
    l=1
    for k in top10_item_title:
      print("item ",l,": ",k)
      l+=1
    

Actual item bought by user:  Avalon Grapefruit and Geranium Smoothing Shampoo, 11 Ounce
Top 10 recommended items for user:  974
item  1 :  Avalon Grapefruit and Geranium Smoothing Shampoo, 11 Ounce
item  2 :  FusionBeauty LiftFusion Face Lift
item  3 :  Clean &amp; Clear Deep Action Cream Facial Cleanser for Sensitive Skin, Gentle Daily Face Wash with Oil-Free, 6.5 oz (Pack of 4)
item  4 :  Clean &amp; Clear Deep Action Cream Facial Cleanser for Sensitive Skin, Gentle Daily Face Wash with Oil-Free, 6.5 oz (Pack of 4)
item  5 :  Urban Spa Natural Bamboo and Jute Bath Mitt
item  6 :  Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz
item  7 :  Pre de Provence Artisanal French Soap Bar Enriched with Shea Butter, Quad-Milled For A Smooth &amp; Rich Lather (150 grams) - Raspberry
item  8 :  Pre de Provence Artisanal French Soap Bar Enriched with Shea Butter, Quad-Milled For A Smooth &amp; Rich Lather (150 grams) - Raspberry
item  9 :  16
item  10 :  27


**Nesterov optimizer along with momentum 0.09**

In [ ]:
class NCF(pl.LightningModule):

    def __init__(self, num_users, num_items, df, all_pids):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.df = df
        self.all_pids = all_pids
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.LeakyReLU()(self.fc1(vector))
        vector = nn.LeakyReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(),lr=0.05,nesterov=True,momentum=0.09,dampening=0)

    def train_dataloader(self):
        return DataLoader(AmazonTrainDataset(self.df, self.all_pids),
                          batch_size=512, num_workers=4)

In [ ]:
num_users = len(df['reviewerID'])
num_items = len(df['asin'])
all_pids = df['asin'].unique()

model = NCF(num_users, num_items, train_data, all_pids)

trainer = pl.Trainer(max_epochs=200, gpus=1, reload_dataloaders_every_epoch=True,
                     progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 42.2 K
1 | item_embedding | Embedding | 42.2 K
2 | fc1            | Linear    | 1.1 K 
3 | fc2            | Linear    | 2.1 K 
4 | output         | Linear    | 33    
---------------------------------------------
87.5 K    Trainable params
0         Non-trainable params
87.5 K    Total params
0.350     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze

1

In [ ]:
# User-item pairs for testing
test_user_item_set = set(zip(test_data['reviewerID'], test_data['asin']))

# Dict of all items that are interacted with by each user
user_interacted_items = df.groupby('reviewerID')['asin'].apply(list).to_dict()

hits = []
for (u,i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_pids) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
  
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))

The Hit Ratio @ 10 is 0.80


In [ ]:
top10_item_title = []
for (u,i) in [((974, 40))]:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_pids) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
  
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    for j in top10_items:
      try:
        top10_item_title.append(get_product(test_data[test_data['asin']==j]['asin'].map(e).iloc[0])['title'])
      except:
        top10_item_title.append(j)
    print("Actual item bought by user: ",get_product(test_data[test_data['asin']==i]['asin'].map(e).iloc[0])['title'])
    print("Top 10 recommended items for user: ",u)
    l=1
    for k in top10_item_title:
      print("item ",l,": ",k)
      l+=1

Actual item bought by user:  Avalon Grapefruit and Geranium Smoothing Shampoo, 11 Ounce
Top 10 recommended items for user:  974
item  1 :  Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz
item  2 :  Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz
item  3 :  Avalon Grapefruit and Geranium Smoothing Shampoo, 11 Ounce
item  4 :  Pre de Provence Artisanal French Soap Bar Enriched with Shea Butter, Quad-Milled For A Smooth &amp; Rich Lather (150 grams) - Raspberry
item  5 :  Pre de Provence Artisanal French Soap Bar Enriched with Shea Butter, Quad-Milled For A Smooth &amp; Rich Lather (150 grams) - Raspberry
item  6 :  77
item  7 :  77
item  8 :  Urban Spa Moisturizing Booties to Keep your Feet Smooth, Hydrated and Moisturized
item  9 :  Urban Spa Moisturizing Booties to Keep your Feet Smooth, Hydrated and Moisturized
item  10 :  Urban Spa Moisturizing Booties to Keep your Feet Smooth, Hydrated and Moisturized


**RMSprop optimizer**

In [ ]:
class NCF(pl.LightningModule):

    def __init__(self, num_users, num_items, df, all_pids):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=128)
        self.output = nn.Linear(in_features=128, out_features=1)
        self.df = df
        self.all_pids = all_pids
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.LeakyReLU()(self.fc1(vector))
        vector = nn.LeakyReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters())

    def train_dataloader(self):
        return DataLoader(AmazonTrainDataset(self.df, self.all_pids),
                          batch_size=512, num_workers=4)

In [ ]:
num_users = len(df['reviewerID'])
num_items = len(df['asin'])
all_pids = df['asin'].unique()

model = NCF(num_users, num_items, train_data, all_pids)

trainer = pl.Trainer(max_epochs=200, gpus=1, reload_dataloaders_every_epoch=True,
                     progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 42.2 K
1 | item_embedding | Embedding | 42.2 K
2 | fc1            | Linear    | 1.1 K 
3 | fc2            | Linear    | 8.3 K 
4 | output         | Linear    | 129   
---------------------------------------------
93.8 K    Trainable params
0         Non-trainable params
93.8 K    Total params
0.375     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze

1

In [ ]:
# User-item pairs for testing
test_user_item_set = set(zip(test_data['reviewerID'], test_data['asin']))

# Dict of all items that are interacted with by each user
user_interacted_items = df.groupby('reviewerID')['asin'].apply(list).to_dict()

hits = []
for (u,i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_pids) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
  
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    #print("Top 10 recommended items for user: ",u,"is :",top10_items)
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))

The Hit Ratio @ 10 is 0.90


In [ ]:
top10_item_title = []
for (u,i) in [((974, 40))]:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_pids) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
  
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    for j in top10_items:
      try:
        top10_item_title.append(get_product(test_data[test_data['asin']==j]['asin'].map(e).iloc[0])['title'])
      except:
        top10_item_title.append(j)
    print("Actual item bought by user: ",get_product(test_data[test_data['asin']==i]['asin'].map(e).iloc[0])['title'])
    print("Top 10 recommended items for user: ",u)
    l=1
    for k in top10_item_title:
      print("item ",l,": ",k)
      l+=1

Actual item bought by user:  Avalon Grapefruit and Geranium Smoothing Shampoo, 11 Ounce
Top 10 recommended items for user:  974
item  1 :  Avalon Grapefruit and Geranium Smoothing Shampoo, 11 Ounce
item  2 :  Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz
item  3 :  Fruits &amp; Passion Blue Refreshing Shower Gel - 6.7 fl. oz.
item  4 :  65
item  5 :  77
item  6 :  16
item  7 :  68
item  8 :  14
item  9 :  12
item  10 :  14
